# Homework 1 - Selen Parlar Özçelik
The goal of this homework is to train a simple model for predicting the duration of a ride.


In [ ]:
### Download NYC Yellow Taxi Data
! wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet -P ./data
! wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet -P ./data

In [7]:
### Import libraries
import pandas as pd

from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [8]:
### Load data
df_jan = pd.read_parquet('./data/yellow_tripdata_2023-01.parquet')
df_feb = pd.read_parquet('./data/yellow_tripdata_2023-02.parquet')

## Homework
### 1. Read the data for January. How many columns are there?

In [10]:
# print(f"Size of January DataFrame: {df_jan.shape}")
print(f"> There are {df_jan.shape[1]} columns in January DataFrame")
df_jan.head(5)

> There are 19 columns in January DataFrame


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


### 2. What's the standard deviation of the trips duration in January?

In [ ]:
df_jan.info()
def calculate_duration(df):
    """
    Adds a 'duration' column to the DataFrame, calculated as the difference between
    'tpep_dropoff_datetime' and 'tpep_pickup_datetime', converted to minutes.

    Args:
        df (pd.DataFrame): Input DataFrame with 'tpep_pickup_datetime' and 'tpep_dropoff_datetime' columns.

    Returns:
        pd.DataFrame: DataFrame with an additional 'duration' column.
    """
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df['duration'].apply(lambda td: td.total_seconds() / 60)
    return df
# Add a new column 'duration' to DataFrames
df_jan = calculate_duration(df_jan)
df_feb = calculate_duration(df_feb)
# Calculate std of duration column
print(f"> Standard deviation of duration in January: {df_jan.duration.std() :.2f} minutes")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [ ]:
### 3. What fraction of the records left after you dropped the outliers?

In [ ]:
# We need to check the distribution of the duration variable. There are some outliers. 
def filter_duration(df):
    """
    Filters the DataFrame to keep only records with a duration between 1 and 60 minutes.
    Args:
        df (pd.DataFrame): Input DataFrame with a 'duration' column.
    Returns:
        pd.DataFrame: Filtered DataFrame with records having duration between 1 and 60 minutes.
    """
    return df[(df.duration >= 1) & (df.duration <= 60)]

df_jan_filtered = filter_duration(df_jan)
df_feb_filtered = filter_duration(df_feb)

# Fraction of records after dropping outliers
print(f"> Fraction of records after dropping outliers: {(df_jan_filtered.shape[0] / df_jan.shape[0]) * 100:.2f}%")


> Fraction of records after dropping outliers: 98.12%


### 4. What's the dimensionality of this matrix (number of columns)?

In [13]:
# Let's apply one-hot encoding to the pickup and dropoff location IDs.
categorical_features = ['PULocationID', 'DOLocationID']
numerical_feature = ['trip_distance']

# Convert int64 to str so that one-hot encoding works - since OHE takes everything as categorical which is not int64
df_jan_filtered[categorical_features] = df_jan_filtered[categorical_features].astype(str)
df_feb_filtered[categorical_features] = df_feb_filtered[categorical_features].astype(str)

# Apply one-hot encoding:
# We can use DictVectorizer or OneHotEncoder. Here we will use OneHotEncoder
# OneHotEncoder is more efficient for large datasets
# and it can handle unknown categories in the test set
one_hot_encoder = OneHotEncoder(handle_unknown='ignore')
transformed_features = one_hot_encoder.fit_transform(df_jan_filtered[categorical_features])

print(f"> Dimentionality of transformed features: {transformed_features.shape[1]}")

/tmp/ipykernel_18754/2320927833.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_jan_filtered[categorical_features] = df_jan_filtered[categorical_features].astype(str)
/tmp/ipykernel_18754/2320927833.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_feb_filtered[categorical_features] = df_feb_filtered[categorical_features].astype(str)


> Dimentionality of transformed features: 515


### 5. What's the RMSE on train?


In [ ]:
# Train a plain linear regression model with default parameters, where duration is the response variable
training_targets = df_jan_filtered['duration']
lr = LinearRegression()
lr.fit(transformed_features, training_targets)
# Calculate the RMSE of the model on the training data
training_predictions = lr.predict(transformed_features)
rmse = mean_squared_error(training_targets, training_predictions, squared=False)
print(f"> RMSE of the model on the training data: {rmse:.2f} minutes")

### 6. What's the RMSE on validation?


In [ ]:
# Apply this model to the validation dataset (February 2023).
test_targets = df_feb_filtered['duration']

# Transform the test set using the same one-hot encoder
transformed_test_features = one_hot_encoder.transform(df_feb_filtered[categorical_features])

# Predict the duration using the model
test_predictions = lr.predict(transformed_test_features)

# Calculate the RMSE of the model on the test data
rmse_test = mean_squared_error(test_targets, test_predictions, squared=False)

# Print the RMSE of the model on the test data
print(f"> RMSE of the model on the test data: {rmse_test:.2f} minutes")